# 自动求导机制 Autograd

In [69]:
# 张量创建时，通过设置 requires_grad 标识为Ture告诉Pytorch需要对该张量进行自动求导
# PyTorch会记录该张量的每一步操作历史并自动计算
import torch
x = torch.rand(2,3, requires_grad=True)
y = torch.rand(2,3, requires_grad=True)
x,y

(tensor([[0.7683, 0.4898, 0.1517],
         [0.5925, 0.3149, 0.3559]], requires_grad=True),
 tensor([[0.7724, 0.8816, 0.1016],
         [0.5211, 0.3003, 0.4774]], requires_grad=True))

In [75]:
z = torch.sum(x+y) # x+y是个2*3的矩阵 sum之后就是x11+y11+x12+y12+...+x23+y23 那么对这六个xi求偏导 必然全为1
z

tensor(5.7276, grad_fn=<SumBackward0>)

Tensor进行操作后，grad_fn已经被赋予了一个新的函数

PyTorch会自动追踪记录与张量的所有操作，计算完成后调用.backward()方法自动计算梯度并且将结果保存到grad属性中 \\

## 1. 标量的反向传播

如果Tensor类表示的是一个标量scaler,则不需要为backward()指定任何参数;

In [76]:
z.backward() # 自动计算梯度 并将计算结果保存到grad属性中
x.grad, y.grad # x.grad就是指z对x的偏导

(tensor([[2.5366, 1.9796, 1.3034],
         [2.1850, 1.6297, 1.7119]]),
 tensor([[2.7897, 3.3316, 1.0310],
         [1.8147, 1.2706, 1.6837]]))

## 2. 非标量的反向传播

但是如果Tensor有更多的元素，是个向量或矩阵，则需要指定一个gradient参数，它是形状匹配的张量

In [77]:
# 在默认情况下，PyTorch会累积梯度，我们需要清除之前的值
x.grad.zero_()
y.grad.zero_()
z = x**2 + y**3
x,y

(tensor([[0.7683, 0.4898, 0.1517],
         [0.5925, 0.3149, 0.3559]], requires_grad=True),
 tensor([[0.7724, 0.8816, 0.1016],
         [0.5211, 0.3003, 0.4774]], requires_grad=True))

In [78]:
# 我们的返回值不是一个标量，所以需要输入一个大小相同的张量作为参数，这里我们用ones_like函数根据x生成一个张量
z.backward(torch.ones_like(x)) # z.sum().backward()同效
x.grad, y.grad # 即2*x 与 3y^2

(tensor([[1.5366, 0.9796, 0.3034],
         [1.1850, 0.6297, 0.7119]]),
 tensor([[1.7897, 2.3316, 0.0310],
         [0.8147, 0.2706, 0.6837]]))

通过.is_leaf判断变量时创建变量还是结果变量

In [15]:
print('x.is_leaf =', x.is_leaf) # 创建变量 手动创建； 是叶子节点
print('y.is_leaf =', y.is_leaf)
print('z.is_leaf =', z.is_leaf) # 结果变量 运算得到

x.is_leaf = True
y.is_leaf = True
z.is_leaf = False


In [16]:
z.grad_fn.next_functions

((<PowBackward0 at 0x1a78e0bca00>, 0), (<PowBackward0 at 0x1a78e0bcac0>, 0))